In [2]:
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding
import datasets

In [2]:
raw_datasets = datasets.load_dataset('glue', 'mrpc')

Found cached dataset glue (/Users/shuyuzhou/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from datasets import load_metric
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds.predictions, eval_preds.label_ids
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [4]:
# checkpoint = 'roberta-large'
# checkpoint = 'facebook/bart-large'
# checkpoint = './models/gpt2-glue-tokenizer/'

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(sample):
    return tokenizer(sample['sentence1'], sample['sentence2'], truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


from transformers import AutoModelForSequenceClassification
# model_checkpoint = "./models/bart_0.1/"
# model_checkpoint = "./models/roberta_0.5/"
# model_checkpoint = "./models/gpt2-glue_0.5/"
model_checkpoint = 'bert-base-uncased'
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)


from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    num_train_epochs = 1,
    output_dir='output',
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator, 
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Loading cached processed dataset at /Users/shuyuzhou/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-31906a1957e9dbf7.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /Users/shuyuzhou/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-54149c91d00919a2.arrow
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (

In [5]:
x = trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2. If sentence1, idx, sentence2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3668
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 459
  Number of trainable parameters = 109483778


  0%|          | 0/459 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 1985.4646, 'train_samples_per_second': 1.847, 'train_steps_per_second': 0.231, 'train_loss': 0.4770612529679841, 'epoch': 1.0}


In [6]:
# for info in x:
#     print(info)

x.metrics

{'train_runtime': 1985.4646,
 'train_samples_per_second': 1.847,
 'train_steps_per_second': 0.231,
 'train_loss': 0.4770612529679841,
 'epoch': 1.0}

In [60]:
predictions = trainer.predict(tokenized_datasets['validation'])
print(predictions.metrics)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: idx, sentence1, sentence2. If idx, sentence1, sentence2 are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1725
  Batch size = 8


  0%|          | 0/216 [00:00<?, ?it/s]

{'test_loss': 0.4371364116668701, 'test_runtime': 201.1947, 'test_samples_per_second': 8.574, 'test_steps_per_second': 1.074}


In [63]:
import json
 
# Data to be written
results = x.metrics
results.update(predictions.metrics)
print(results)
 
# Serializing json
json_object = json.dumps(results, indent=4)
 
# Writing to sample.json
with open("output/sample.json", "w") as outfile:
    outfile.write(json_object)

{'train_runtime': 2186.2738, 'train_samples_per_second': 1.678, 'train_steps_per_second': 0.21, 'train_loss': 0.525709463879953, 'epoch': 1.0, 'test_loss': 0.4371364116668701, 'test_runtime': 201.1947, 'test_samples_per_second': 8.574, 'test_steps_per_second': 1.074}


## CLM

In [3]:
raw_datasets = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')

Found cached dataset wikitext (/Users/shuyuzhou/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
context_length = 128

# checkpoint = 'roberta-large'
# checkpoint = 'gpt2'
checkpoint = 'facebook/bart-large'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(samples):
    outputs = tokenizer(
        samples["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize_function, batched=True, remove_columns=raw_datasets["train"].column_names
)


from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/37 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [5]:
from transformers import AutoConfig, AutoModelForCausalLM
# model_checkpoint = 'roberta-large'
# model_checkpoint = 'gpt2'
model_checkpoint = 'facebook/bart-large'


config = AutoConfig.from_pretrained(
    model_checkpoint,
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = AutoModelForCausalLM.from_config(config)


from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='output',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs = 1,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [6]:
x = trainer.train()

/opt/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10555
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 330
  Number of trainable parameters = 254084096


  0%|          | 0/330 [00:00<?, ?it/s]

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Summarization

In [71]:
raw_datasets = datasets.load_dataset('xsum')
raw_datasets = raw_datasets.remove_columns(['id'])

Using custom data configuration default


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /Users/shuyuzhou/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [73]:
checkpoint = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["document"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["summary"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels_mask"] = labels["attention_mask"]
    return model_inputs


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=['document', 'summary'])


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/shuyuzhou/.cache/huggingface/hub/models--gpt2/snapshots/75e09b43581151bd1d9ef6700faa605df408979f/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {


  0%|          | 0/205 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [78]:
from transformers import AutoModelForSeq2SeqLM

model_checkpoint = 'roberta-large'
# model_checkpoint = 'gpt2'
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

loading configuration file config.json from cache at /Users/shuyuzhou/.cache/huggingface/hub/models--roberta-large/snapshots/5069d8a2a32a7df4c69ef9b56348be04152a2341/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



ValueError: Unrecognized configuration class <class 'transformers.models.roberta.configuration_roberta.RobertaConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, T5Config, XLMProphetNetConfig.